## Import and initialize statements from 'Get started'

*(Select a cell and press `ctrl` + `Enter` to run it.)*

In [9]:
import time

from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.io import fits
from astropy.table import Table, Column
from astropy.time import Time, TimeDelta # note capitalization
from astropy.wcs import WCS

import numpy as np

from astroquery.nasa_exoplanet_archive import NasaExoplanetArchive
from astroquery.skyview import SkyView

from pywwt.jupyter import WWTJupyterWidget

In [19]:
wwt = WWTJupyterWidget()
wwt

## Plot K2’s footprint and exoplanet yield.

**First, we use a class from the `astroquery` package to query NASA's Exoplanet Archive and return an `astropy` table that contains information about every confirmed exoplanet so far. It's a large table, so we'll only keep the columns necessary to run this example.**

*(If the `astroquery` call doesn't load, uncomment and run the cell following it to download the table from a `pywwt`-affiliated GitHub repository.)*

In [ ]:
xarch = NasaExoplanetArchive.get_confirmed_planets_table()
xarch.keep_columns(['pl_hostname', 'pl_letter', 'st_dist', 'ra', 'dec'])

In [ ]:
#xarch = Table.read('https://raw.githubusercontent.com/WorldWideTelescope/pywwt-notebooks/master/tutorials/data/k2_table.ecsv',
#                   format='ascii.ecsv')

*(Open a new cell above this one and see what the table looks like by entering `xarch` and running it.)*

**Next, we slice the table twice so it only contains entries of 'b' planets observed by K2.** 

K2 planets and stars are named using a consistent system: 

```K2 + 'system number' + 'letter'```. 

The letter for a star is typically 'a', the first exoplanet confirmed in the system is 'b', and so on.

We are plotting stars, but since the table only contains exoplanets (letters from 'b' onward), we will slice the table so only planets with the `b` suffix remain. That gives us one entry per system.

**`k2_pl` searches the `xarch` column that lists star names and only keeps those with K2 at the beginning. Then, `k2_st` searches the suffix of the planet name and only keeps `b` planets.**

*(If you aren't familiar with list comprehensions, see [Note 1](aas233_tutorial.ipynb#Note-1).)*

In [ ]:
# select only k2 planets
k2_pl = xarch[ [row.startswith('K2') for row in xarch['pl_hostname']] ]

# select only 'b' planets from k2
k2_st = k2_pl[ ['b' in row for row in k2_pl['pl_letter']] ]

**After that, we load the footprint of the K2 campaign fields on the sky using pywwt's `add_fov` method.**

*(Loading K2's footprint usually takes a minute, so please be patient with this step.)*

In [ ]:
field = wwt.add_fov(wwt.instruments.k2)

**Move the viewer down here and watch what happens next.**

**Once the footprint has loaded, we finally create the data layer that will plot our points in the viewer.**

`pywwt` reads the locations from the keyword arguments `lon_att` and `lat_att`, and we feed it the names of the columns in `k2_st` that contain the stellar right ascensions and declinations. We can also set other attributes like color and size in the function call.

In [ ]:
lay = wwt.layers.add_table_layer(table=k2_st, frame='Sky',
                                lon_att='ra', lat_att='dec',
                                color='#c4d600', size_scale=40)

Do you see the points? If not, try making them larger.

In [ ]:
lay.size_scale *= 2

If you want to see the points and footprint without a sky background, `pywwt` also has a black layer available.

In [ ]:
wwt.foreground_opacity = 0
wwt.background = wwt.imagery.other.black

**It is also possible to visualize data layers in `pywwt`'s 3D solar system mode. To do so, we'll switch the view.**

We will also remove the K2 footprint, as `add_fov` is designed to work in 2D.

In [ ]:
field.remove()
wwt.set_view('solar system')

**Bring the viewer down here, zoom out, and you'll see K2's exoplanet-hosting stars in 3D. All of them are currently the same distance from Earth -- let's fix that.**

Data layers also have attributes related to distance and altitude. We set `alt_type` to the correct type for our scenario, and let `pywwt` know which column of the table contains distance with `alt_att`.

In [ ]:
lay.alt_type = 'distance'
lay.alt_att = 'st_dist'

**Zoom out again and see the stars at their proper distances.**

## Layer FITS images over existing all-sky surveys.

**We again use `astroquery` to get our data, this time harnessing the `SkyView` service to retrieve a FITS cutout of a portion of M101 that experienced a Type Ia supernova in 2011.**

The cutout comes from a pre-event image from the highest wavelength band of 2MASS, an infrared all-sky survey. `SkyView` can also access of ther bands of 2MASS and many other surveys, as shown in [Note 2](aas233_tutorial.ipynb#Note-2).

*(Again, if the `astroquery` call doesn't load, uncomment and run the cell following it to download the table from a `pywwt`-affiliated GitHub repository.)*

In [2]:
size = 500 # choose the size of the image in pixels
img_results = SkyView.get_images(position='SN 2011FE',
                                 survey='2MASS-K', pixels=500)

In [ ]:
# MERGE PR WITH DATA, THEN CHANGE LINK
#img_results = [fits.open('https://raw.githubusercontent.com/...')]

In [7]:
img_results = sn11.copy()

Once we have the image data, we can take a look at it. Because `SkyView.get_images()` returns a list, we have to index `img_results` to get to the FITS image data itself. In this example, `img_results` is a list of one.

In [8]:
sn11 = img_results[0]
sn11.info()

Filename: (No file associated with this HDUList)
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     141   (500, 500)   float32   


Next, we look at the `PrimaryHDU`, or "Header Data Unit," in more detail. In consists of a header and the actual image data, which we'll save as separate variables.

In [13]:
sn11_data = sn11[0].data
sn11_header = sn11[0].header

`pywwt` requires the image data to be structured as a 2-D array, which is what we see below.  (The dimensions here are (`size` x `size`), just as in the earlier cell that displayed output from `sn11.info()`).

In [17]:
print(sn11_data.shape)
sn11_data

(500, 500)


array([[359.08322, 358.65552, 357.8507 , ..., 357.51172, 357.50912,
        357.82022],
       [358.6935 , 358.98737, 358.19406, ..., 357.9415 , 357.47458,
        357.56094],
       [358.53577, 359.668  , 359.38986, ..., 357.59906, 356.78537,
        357.1075 ],
       ...,
       [359.20923, 359.32407, 358.82938, ..., 360.84332, 360.7469 ,
        360.21722],
       [359.30783, 360.0317 , 359.4415 , ..., 360.42014, 361.19058,
        361.80167],
       [359.7121 , 360.7487 , 360.17764, ..., 359.50308, 360.78357,
        361.92316]], dtype=float32)

The header contains base level data about the image. We will need it to create an `astropy.wcs.WCS` object, which `pywwt` uses to convert the image to its preferred coordinate system and orient the data properly on its 2-D sky projection.

In [14]:
sn11_header

SIMPLE  =                    T / Written by SkyView Wed Jun 05 17:36:38 EDT 2019
BITPIX  =                  -32 / 4 byte floating point                          
NAXIS   =                    2 / Two dimensional image                          
NAXIS1  =                  500 / Width of image                                 
NAXIS2  =                  500 / Height of image                                
CRVAL1  =              210.774 / Reference longitude                            
CRVAL2  =              54.2737 / Reference latitude                             
RADESYS = 'FK5     '           / Coordinate system                              
EQUINOX =               2000.0 / Epoch of the equinox                           
CTYPE1  = 'RA---TAN'           / Coordinates -- projection                      
CTYPE2  = 'DEC--TAN'           / Coordinates -- projection                      
CRPIX1  =                250.5 / X reference pixel                              
CRPIX2  =                250

Speaking of which, let's create the WCS (World Coordinate System) object now. As stated before, all we need is the header data; `astropy`'s internals do the rest of the heavy lifting.

In [18]:
WCS(sn11_header)

WCS Keywords

Number of WCS axes: 2
CTYPE : 'RA---TAN'  'DEC--TAN'  
CRVAL : 210.774  54.2737  
CRPIX : 250.5  250.5  
PC1_1 PC1_2  : 1.0  0.0  
PC2_1 PC2_2  : 0.0  1.0  
CDELT : -0.0002777777  0.0002777777  
NAXIS : 500  500

At this point, we have everything we need to create an image layer, so move the `wwt` cell down to this section and re-execute it to refresh the viewer state. Then, since we're working with an IR image, we'll switch the background to match that. Finally, we make the foreground layer invisible for the sake of simplicity.

In [20]:
wwt.background = wwt.imagery.ir.twomass # do other IR surveys look good?
wwt.foreground_opacity = 0

**The time has come for us to load the image into pywwt.**

When working directly with separate header and data objects as we've been, the `add_image_layer` method requires a tuple argument as shown below. `pywwt` will then take the viewer to the proper location and scale to see your image.

In [ ]:
lay = wwt.layers.add_image_layer(image=(sn11_data, WCS(sn11_header)))

(_When the header and data are wrapped into one HDU object, we can also upload the image like so:_

`wwt.layers.add_image_layer(sn11)`

_It's also possible to upload FITS images from a local file by using `the/path/to/your/file` as the method's argument instead of an existing `Python` object name. `pywwt` will then automatically do the work of finding the header and data that we did in previous cells._)

Once we see the image, we are able to edit a few of its attributes: `opacity`, `vmin`/`vmax` (the minimum and maximum flux values covered by the colormap), and `stretch` (the scaling function used to display the data).

Changing opacity shows us that our image is indeed aligned with the background survey.

In [ ]:
lay.opacity = .4

In [ ]:
lay.opacity = .8

Shifting `vmin` and `vmax` changes the lower and upper bounds of our grayscale colormap.

`pywwt` automatically sets these attributes such that ~99% of the data is covered, but we can adjust it depending on what exactly we'd like to emphasize in our image.

In [22]:
print(np.percentile(sn11_data, .5), np.percentile(sn11_data, 99.5))
print(lay.vmin, lay.vmax)

356.3546423339844 365.4453846740723


In [ ]:
lay.vmin = 360

In [ ]:
lay.vmax = 380

Valid `stretch` values are: `linear`, `log`, `power`, `sqrt`, and `histeq`. `pywwt` chooses a linear stretch by default, but again, we can adjust it to fit our individual needs -- aesthetic, scientific, or anywhere in between.

In [ ]:
lay.stretch = 'power'

**The great advantage of being able to upload personal FITS images into `pywwt` is that we can easily compare them with multiple surveys at once.**

Since our FITS image is in infrared, we can use background all-sky surveys from `pywwt` to see M101 in three wavelengths at once. (Please give them a few seconds to load.)

In [23]:
wwt.background = wwt.imagery.visible.sdss
wwt.foreground = wwt.imagery.gamma.fermi
wwt.foreground_opacity = .7

Finally, we can use `SkyCoord` to get the exact coordinates of the supernova and `pywwt`'s ability to add shapes to the viewer to indicate the spot where the event happened.

In [ ]:
sn11_coord = SkyCoord.from_name('Sn 2011FE')
circ = wwt.add_circle(sn11_coord, radius=.003*u.deg)

As before, we can change attributes of the shape even after its creation, perhaps to even try to simulate how the supernova would have looked as it was happening.

In [ ]:
circ.fill = True

*(possible foray into time series simulations)*

***Is it working? If so, congratulations, enjoy the view, and stay in touch! Find us at the AAS booth, discuss with others at the [WWT forum](https://wwt-forum.org/), and contribute ideas, suggestions, and fixes at our [GitHub repository](https://github.com/WorldWideTelescope/pywwt).***

#### Note 1
*The constructions used to build `k2_pl` and `k2_st` are called "list comprehensions," and they are shortcuts for building lists. The regular, equivalent method to create `k2_st`, looks like this:*

```
index_list = []
for i, row in enumerate(xarch, 0):
    if row['pl_hostname'].startswith('K2'):
        index_list.append(i)

k2_pl = xarch[index_list]
```

#### Note 2

You can see all of the layers available layers in `SkyView` by running `SkyView.list_surveys(`) or `SkyView.survey_dict()`, but be aware that these will take some time to load.

To get links to image files from all bands of a particular survey, run:

`SkyView.get_image_list('Sn 2011FE', survey=SkyView.survey_dict['IR:2MASS class='])`

You can substitute the dictionary entry for whichever surveys are present in `SkyView.survey_dict()`. For example, if you wanted images from all bands of SDSS, the `survey` keyword argument in the call above would change to `SkyView.survey_dict['Optical:SDSS']`